# SQL

**Цели исследования:** 

- Проанализировать базу данных c информацией о книгах, издательствах, авторах, а также пользовательских обзоров книг; 
- Ответить на поставленные вопросы;
- Сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Загрузка и обзор данных

### Таблица `books`

Содержит данные о книгах:

In [4]:
query = '''SELECT *
           FROM books
           LIMIT 5;
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

### Таблица `authors`

Содержит данные об авторах

In [5]:
query = '''SELECT *
           FROM authors
           LIMIT 5;
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


- `author_id` — идентификатор автора;
- `author` — имя автора.

### Таблица `publishers`

Содержит данные об издательствах:

In [6]:
query = '''SELECT *
           FROM publishers
           LIMIT 5;
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

### Таблица `ratings`

Содержит данные о пользовательских оценках книг

In [7]:
query = '''SELECT *
           FROM ratings
           LIMIT 5;
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

### Таблица `reviews`

Содержит данные о пользовательских обзорах на книги

In [8]:
query = '''SELECT *
           FROM reviews
           LIMIT 5;
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Сколько книг вышло после 1 января 2000 года

In [9]:
query = '''SELECT COUNT(book_id) AS cnt_books
           FROM books
           WHERE publication_date > '2000-01-01';
            '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,cnt_books
0,819


После 1 января 2000г вышло 819 книг

## Посчитать количество обзоров и среднюю оценку для каждой книги

In [10]:
query = '''SELECT b.book_id,
                  b.title,
                  COUNT(DISTINCT re.review_id) AS cnt_reviews,
                  ROUND(AVG(ra.rating),1) AS avg_rating
           FROM books b
           LEFT JOIN ratings ra ON b.book_id=ra.book_id
           LEFT JOIN reviews re ON b.book_id=re.book_id
           GROUP BY 1,2
           ORDER BY 3 DESC;
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.7
1,963,Water for Elephants,6,4.0
2,734,The Glass Castle,6,4.2
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
4,695,The Curious Incident of the Dog in the Night-Time,6,4.1
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.7
996,808,The Natural Way to Draw,0,3.0
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.0
998,221,Essential Tales and Poems,0,4.0


Больше всего ревью получила книга `Twilight (Twilight #1)` и срднюю оценку 3.7.

## Определить издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query = '''SELECT p.publisher,
                  COUNT(b.book_id) AS cnt_books                
           FROM books b
           INNER JOIN publishers p ON b.publisher_id=p.publisher_id
           WHERE b.num_pages>50
           GROUP BY 1
           ORDER BY 2 DESC
           LIMIT 3;
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,publisher,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


Издательство `Penguin Books` выпустило наибольшее число книг.

## Определить автора с самой высокой средней оценкой книг (учитывайть только книги с 50 и более оценками)

In [12]:
query = '''SELECT a.author,
                  ROUND(AVG(ra.rating),2) AS avg_rating
           FROM books b
           JOIN authors a ON b.author_id = a.author_id
           JOIN ratings ra ON b.book_id = ra.book_id
           WHERE b.book_id IN ( SELECT book_id
                                FROM ratings
                                GROUP BY 1
                                HAVING COUNT(rating_id)>=50)
           GROUP BY a.author
           ORDER BY 2 DESC;
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.25
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,William Shakespeare/Paul Werstine/Barbara A. M...,3.79
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
9,Lois Lowry,3.75


J.K. Rowling/Mary GrandPré авторы с наивысшей средней оценкой, среди книг, у которых больше 50 оценок

## Посчитать среднее количество обзоров от пользователей (которые поставили больше 50 оценок)

In [13]:
query = '''SELECT COUNT(DISTINCT re.text) / COUNT(DISTINCT re.username) AS avg_reviews
           FROM ratings ra
           JOIN reviews re ON ra.book_id = re.book_id
           WHERE re.username IN ( SELECT username
                                FROM ratings
                                GROUP BY 1
                                HAVING COUNT(rating_id)>50
                               );
        '''

query = pd.io.sql.read_sql(query, con = engine) 
query

,avg_reviews
0,24


Активные пользователи (которые поставили больше 50 оценок) в среднем пишут 24 ревью.

## Вывод

- С 2000 г было выпущено 819 книг;
- Лидером по количеству отзывов стала книга `Twilight (Twilight #1)` и получила среднюю оценку 3.7;
- Издательство выпустившее наибольшее число книг - `Penguin Books` 42шт;
- `J.K. Rowling/Mary GrandPré` авторы с наивысшей средней оценкой, среди книг, у которых больше 50 оценок;
- Активные пользователи (которые поставили более 50 оценок) в среднем пишут 24 ревью, т.е. примерно половину от выставленных оценок.